In [1]:
import tensorflow as tf
tf.test.gpu_device_name()



'/device:GPU:0'

In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 120kB/s 
     |████████████████████████████████| 6.8MB 19.1MB/s 
     |████████████████████████████████| 983kB 45.9MB/s 
     |████████████████████████████████| 266kB 48.5MB/s 
     |████████████████████████████████| 1.9MB 38.9MB/s 
     |████████████████████████████████| 1.2MB 42.0MB/s 
     |████████████████████████████████| 471kB 43.7MB/s 
     |████████████████████████████████| 901kB 46.2MB/s 
     |████████████████████████████████| 3.3MB 42.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.3-cp37-none-any.whl size=25282390 sha256=9b70e10fda94532c0af88f70eb96755e38a727c5c8671f6ce1f2e32312607823
  Stored in directory: /root/.cache/pip/wheels/16/05/be/d6e659b3349016b1059e19fa028f165af4eeae2c196f329112
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=1d42a6571b6996f5748285861db0b98b54f69ca23c2ab044ab1c074cbc089a44
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [3]:
import ktrain

In [4]:

from ktrain import text
import pandas as pd
import json

with open('Musical_Instruments_5.json') as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 

df = pd.DataFrame(data)

In [5]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [6]:
#df["target"]= df["overall"].replace([5.0,4.0,3.0,2.0,1.0],["five","four","three","two","one"])
df["target"]= df["overall"].astype('category').cat.codes

In [7]:
X=list(df['reviewText'])
y=list(df['target'])

In [8]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,target
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014",4
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013",4
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013",4
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014",4
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014",4


In [9]:
df["target"].unique()

array([4, 2, 3, 1, 0], dtype=int8)

In [10]:
df["target"].value_counts()/len(df["target"])*100

4    67.615242
3    20.309911
2     7.523633
1     2.436410
0     2.114804
Name: target, dtype: float64

In [11]:
label_list= ['five', 'three', 'four', 'two', 'one']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
#y_test

In [13]:
#Customised Structure of model is created
model_name = "distilbert-base-uncased"
#model_name= "bert-base-uncased"
#model_name = "joeddav/distilbert-base-uncased-go-emotions-student"
#model_name = "google/tapas-tiny-finetuned-tabfact"
trans= text.Transformer(model_name,maxlen= 512,classes=[4, 2, 3, 1, 0])
trans.preprocess_train_called = True

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:414: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


In [14]:
#Preprocessing of the trained and test data to convert into transformer dataset
train_data= trans.preprocess_train(X_train,y_train)
test_data= trans.preprocess_test(X_test,y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 90
	95percentile : 287
	99percentile : 545


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 91
	95percentile : 275
	99percentile : 549


In [15]:
test_data.nsamples()

2053

In [16]:
#Get the customised model
model = trans.get_classifier()

In [17]:
##Actual Training of customized data on customised model
learner = ktrain.get_learner(model,train_data=train_data,val_data=test_data,batch_size=16)

In [18]:
#learner.lr_find(show_plot=True,max_epochs=3)

In [19]:
learner.fit_onecycle(1e-4,1)



begin training using onecycle policy with max lr of 0.0001...
513/513 [==============================] - 503s 960ms/step - loss: 0.8703 - accuracy: 0.6797 - val_loss: 0.7719 - val_accuracy: 0.6897


In [20]:
learner.validate(class_names=[4, 2, 3, 1, 0])

              precision    recall  f1-score   support

           4       0.33      0.38      0.36        39
           2       0.00      0.00      0.00        55
           3       0.33      0.26      0.29       134
           1       0.43      0.06      0.11       451
           0       0.73      0.97      0.83      1374

    accuracy                           0.69      2053
   macro avg       0.37      0.34      0.32      2053
weighted avg       0.61      0.69      0.61      2053



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[  15,    0,    7,    1,   16],
       [   8,    0,   22,    7,   18],
       [  10,    0,   35,   14,   75],
       [   4,    0,   28,   28,  391],
       [   8,    0,   13,   15, 1338]])

In [21]:
learner.view_top_losses(n= 10,preproc= trans)

----------
id:171 | loss:6.81 | true:4 | pred:0)

----------
id:517 | loss:6.44 | true:4 | pred:0)

----------
id:1349 | loss:6.39 | true:4 | pred:0)

----------
id:796 | loss:5.61 | true:2 | pred:0)

----------
id:1035 | loss:5.43 | true:3 | pred:0)

----------
id:1141 | loss:5.4 | true:4 | pred:0)

----------
id:845 | loss:5.33 | true:2 | pred:0)

----------
id:767 | loss:4.98 | true:4 | pred:0)

----------
id:1974 | loss:4.97 | true:2 | pred:0)

----------
id:1333 | loss:4.96 | true:3 | pred:0)



In [23]:
predictor = ktrain.get_predictor(learner.model,preproc=trans)